In [15]:
import pandas as pd
import numpy as np
from data import TASK3_LABELS

In [18]:
import os
li=os.listdir('data_folder')

In [19]:
['hj' + l for l in li]

['hjbaselines',
 'hjdata',
 'hjdata.zip',
 'hjreadme',
 'hjrequirements.txt',
 'hjscorers',
 'hjsemeval2023task3bundle-v2(1).tar',
 'hjsemeval2023task3bundle-v2(1).tgz']

In [23]:
os.path.join('hj', 'hj')

'hj\\hj'

In [2]:
TASK3_LABELS

['Appeal_to_Authority',
 'Appeal_to_Fear-Prejudice',
 'Appeal_to_Hypocrisy',
 'Appeal_to_Popularity',
 'Appeal_to_Time',
 'Appeal_to_Values',
 'Causal_Oversimplification',
 'Consequential_Oversimplification',
 'Conversation_Killer',
 'Doubt',
 'Exaggeration-Minimisation',
 'False_Dilemma-No_Choice',
 'Flag_Waving',
 'Guilt_by_Association',
 'Loaded_Language',
 'Name_Calling-Labeling',
 'Obfuscation-Vagueness-Confusion',
 'Questioning_the_Reputation',
 'Red_Herring',
 'Repetition',
 'Slogans',
 'Straw_Man',
 'Whataboutism']

In [3]:
def to_matrix(df3):    
    df3 = df3.where(pd.notnull(df3), '')
    labels_matrix = np.zeros((df3.shape[0], len(TASK3_LABELS)),int)
    
    for idx, label in enumerate(TASK3_LABELS):
        labels_matrix[:, idx] = df3.labels.apply(lambda x: 1 if label in x else 0)
    return labels_matrix

In [4]:
# removed false dillema training sze with dev is 185, under 150:
id_fews = [2,3,4,5,7,8,13,16,17,18,21,22]
id2label = {idx:label for idx, label in enumerate(TASK3_LABELS)}
np.array(TASK3_LABELS)[id_fews]

array(['Appeal_to_Hypocrisy', 'Appeal_to_Popularity', 'Appeal_to_Time',
       'Appeal_to_Values', 'Consequential_Oversimplification',
       'Conversation_Killer', 'Guilt_by_Association',
       'Obfuscation-Vagueness-Confusion', 'Questioning_the_Reputation',
       'Red_Herring', 'Straw_Man', 'Whataboutism'], dtype='<U32')

In [5]:
df_mul = pd.read_csv('Test_1en_task3_en.txt', sep='\t', header=None, names=['id','line','labels'])
#df_anchor = pd.read_csv('anchor_small_task3_en.txt', sep='\t', header=None, names=['id','line','labels']) #majority_pred_en.csv
df_rob = pd.read_csv('Test_task3_en_rob.txt', sep='\t', header=None, names=['id','line','labels'])
#df_anchor_s = pd.read_csv('majority_pred_en.csv', sep='\t', header=None, names=['id','line','labels'])
matrix_mul1 = to_matrix(df_mul)
matrix_rob1 = to_matrix(df_rob)

In [26]:
df_mul.iloc[0]

id         3116
line          1
labels    Doubt
Name: 0, dtype: object

In [6]:
sum(matrix_mul1)

array([ 47,  83,   2,  15,   0,   0,  14,   0,   9,  94,  62,  19,  85,
         3, 307, 128,   4,   1,   2, 123,  26,   0,   0])

In [7]:
sum(matrix_rob1)

array([  7,  65,   0,   3,   0,   0,  12,   0,  13,  94,  59,  10,  75,
         1, 287, 101,   0,   0,   0, 103,  19,   0,   0])

In [8]:
sum(matrix_rob1>0)[id_fews]

array([ 0,  3,  0,  0,  0, 13,  1,  0,  0,  0,  0,  0])

In [9]:
sum(matrix_mul1>0)[id_fews]

array([ 2, 15,  0,  0,  0,  9,  3,  4,  1,  2,  0,  0])

In [10]:
matrix_com = np.zeros((df_rob.shape[0], len(TASK3_LABELS)),int)
for idx in range(0,23):
    for j in range(0,matrix_com.shape[0]):
        if idx in id_fews:
            matrix_com[j,idx] = matrix_mul1[j,idx]
        else:
            matrix_com[j,idx] = matrix_rob1[j,idx]

In [11]:
sum(matrix_com == matrix_rob1)

array([910, 910, 908, 898, 910, 910, 910, 910, 898, 910, 910, 910, 910,
       906, 910, 910, 906, 909, 908, 910, 910, 910, 910])

In [12]:
sum(matrix_com == matrix_mul1)

array([868, 834, 910, 910, 910, 910, 892, 910, 910, 806, 853, 889, 876,
       910, 758, 823, 910, 910, 910, 768, 891, 910, 910])

In [13]:
pred_com=pd.Series(list(matrix_com))
series_labels=pred_com.apply(lambda x: [id2label[i] for i in np.where(x==1)[0]])
df_rob['labels_pred']=series_labels.apply(lambda x: ','.join(x))
output_pred_file= 'Test_task3_en.txt'
        
df_rob[['id','line','labels_pred']].to_csv(output_pred_file, sep='\t', header=None, index=None)

In [14]:
# tjek the new
df_com = pd.read_csv(output_pred_file, sep='\t', header=None, names=['id','line','labels'])
matrix_com1 = to_matrix(df_com)
sum(matrix_com1)

array([  7,  65,   2,  15,   0,   0,  12,   0,   9,  94,  59,  10,  75,
         3, 287, 101,   4,   1,   2, 103,  19,   0,   0])